## New irap, New Mona data merge with old for modelling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from scipy.stats import pearsonr
from sklearn.linear_model import LogisticRegression 
import statsmodels.api as sm
from sklearn.ensemble import RandomForestClassifier


/Users/xiaoxiangma/.local/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


#### Combine Irap

In [2]:
Podio = pd.read_excel('../June data source/20200430 Podio ILI.xlsx')
Zendesk = pd.read_excel('../June data source/20200511 Zendesk ILI.xlsx')

In [3]:
Podio.shape, Zendesk.shape

((1191, 22), (757, 22))

In [4]:
1191+757

1948

In [5]:
po = Podio[['Mona Unique Session ID', 'Pipeline', 'Date of Referral', 'First Referral',
       'In IRAP Process', 'Screening Stage', 'Screener',
       'Date of Screening Start', 'Program Eligibilities',
       'Mona Case Type Identification',
       'Vulnerability Threshold', 'Screening Decision',
       'Date of Screening Decision', 'Flags', 'Client Location (Country)',
       'Client Nationality']]

In [6]:
zen = Zendesk[['Mona Unique Session ID', 'Pipeline', 'Date of Referral', 'First Referral',
       'In IRAP Process', 'Screening Stage', 'Screener',
       'Date of Screening Start', 'Program Eligibilities',
       'Mona Case Type Identification',
       'Vulnerability Threshold', 'Screening Decision',
       'Date of Screening Decision', 'Flags', 'Client Location (Country)',
       'Client Nationality']]

In [7]:
new_irap = pd.concat([po,zen])

In [8]:
new_irap.shape

(1948, 16)

In [9]:
d1="Data from IRAP 01072020.xlsx"
path = "../Final Soft Launch Results/Current Source Data/"
old_irap = pd.read_excel (path+d1)

In [10]:
old_irap[['Mona Unique Session ID', 'Pipeline', 'Date of Referral', 'First Referral',
       'In IRAP Process', 'Screening Stage', 'Screener',
       'Date of Screening Start', 'Program Eligibilities',
       'Mona Case Type Identification',
       'Vulnerability Threshold', 'Screening Decision',
       'Date of Screening Decision', 'Flags', 'Client Location (Country)',
       'Client Nationality']]

,Mona Unique Session ID,Pipeline,Date of Referral,First Referral,In IRAP Process,Screening Stage,Screener,Date of Screening Start,Program Eligibilities,Mona Case Type Identification,Vulnerability Threshold,Screening Decision,Date of Screening Decision,Flags,Client Location (Country),Client Nationality
0,ce81140c-7083-4463-90ed-ad36d1a41011.2019-07-2...,Mona,2019-07-25,No,No,Screening Complete,Michaela Gallien,2019-08-12,Vulnerability-Based Relocation,Accurate,Below,Cannot Represent,2019-08-12,NaN,NaN,NaN
1,92952aa434524f549543d3fcc79054fe20190804T02434...,Mona,2019-08-03,No,Yes,Screening Complete,Manal ElKhoury,NaT,Vulnerability-Based Relocation,NaN,NaN,NaN,NaT,NaN,NaN,NaN
2,8a4a87a6eeeb43019d820644a8a7534620190802T07350...,Mona,2019-08-02,No,No,Screening,Tiba Fatli,NaT,Vulnerability-Based Relocation,NaN,NaN,NaN,NaT,NaN,NaN,NaN
3,758240c9578a4e29b66bd1d95de9cdc620190802T21293...,Mona,2019-08-02,No,No,Screening Complete,Tania El Khoury,2019-12-20,Vulnerability-Based Relocation,Accurate,Below,Cannot Represent,2019-12-20,NaN,NaN,NaN
4,55865e62-97b3-4656-b470-7eaaa6a517cf.2019-07-2...,Mona,2019-07-26,No,No,Screening Complete,Tania El Khoury,2019-07-31,Vulnerability-Based Relocation;\nCanadian Priv...,Accurate,Below,Cannot Represent,2019-12-17,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1081,0357fd10-acc1-477e-a6a0-87c891104202.2019-07-2...,Mona,2019-07-25,Yes,No,Screening Complete,Hassan Shiban,2019-07-26,Vulnerability-Based RST and Protection,Accurate,Below,Cannot Represent,2019-08-01,NaN,Lebanon,Syrian
1082,034c1a39-1a5f-482f-9634-b2c01d221c88.2019-07-2...,Mona,2019-07-25,Yes,No,Screening Complete,Hassan Shiban,2019-07-26,Vulnerability-Based RST and Protection,Accurate,NaN,Cannot Represent,NaT,Unresponsive,Lebanon,Syrian
1083,03481148-352b-45f1-bb76-746d124d1956.2019-07-2...,Mona,2019-07-25,Yes,No,Screening Complete,Hassan Shiban,2019-07-26,Vulnerability-Based RST and Protection,Accurate,Below,Cannot Represent,2019-07-26,NaN,Lebanon,Syrian
1084,00fd8bad-8e0d-4eea-b193-c719cd73ac32.2019-07-2...,Mona,2019-07-26,Yes,No,Screening Complete,Hassan Shiban,2019-07-28,Vulnerability-Based RST and Protection,Accurate,Below,Cannot Represent,2019-07-30,NaN,Turkey,Syrian


In [11]:
new_irap['Date of Screening Decision'] = new_irap['Date of Screening Decision'].apply(lambda x: np.datetime64('NaT') if pd.isnull(x) else x)

In [12]:
new_irap['Date of Screening Start'] = new_irap['Date of Screening Start'].apply(lambda x: np.datetime64('NaT') if pd.isnull(x) else x)

In [13]:
old_irap['process_days'] = old_irap['Date of Screening Decision']-old_irap['Date of Screening Start']
old_irap['process_days'] = old_irap['process_days'].astype('timedelta64[D]')

In [14]:
new_irap['process_days'] = new_irap['Date of Screening Decision']-new_irap['Date of Screening Start']
new_irap['process_days'] = new_irap['process_days'].astype('timedelta64[D]')

In [15]:
all_irap = pd.concat([new_irap,old_irap])

In [16]:
all_irap.shape

(3034, 23)

#### Combine Mona

In [18]:
new_mona = pd.read_excel('../June data source/Mona Raw Data 5.21.2020.xlsx')

In [19]:
new_mona = new_mona.rename({'Contact UUID': 'Mona Unique User ID'}, axis=1)

In [20]:
full = pd.merge(all_irap, new_mona, on='Mona Unique User ID',how="outer").dropna(axis=1, how='all')

#### Apply same cleaning logic

In [21]:
def label_process_day_1(row):
    if row['process_days'] >= 1 :
        return 1
    if row['process_days'] == 0 :
        return 0
    else: return np.nan

In [22]:
full['process_day_>=1'] = full.apply (lambda row: label_process_day_1(row), axis=1)


In [23]:
full_clean = full.copy()

In [24]:
full.shape

(16483, 342)

In [25]:
def changeto_binary_field(df, column_name):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    
    df[column_name] = df[column_name].apply(lambda x: 0 if pd.isnull(x) else 1)
    print("[log]: Updated to binary null/not null")

In [26]:
for i in ['Field:Aggr 3 Reason Unable to Move',
'Field:I130 Info Rating',
'Field:Aggr 3 Mst Svr Threat',
'Field:DAPI130 Info Rating',
'Field:Aggr 2 Mst Svr Threat',
'Field:Aggr 3 Mst Svr Only Threats',
'Field:P3 Info Rating',
'Field:Aggr 3 Mst Svr Attack',
'Field:Aggr 3 Attack Within Last 6 Months',
'Field:Aggr 2 Reason Unable to Move',
'Field:Aggr 3 Threat Within Last 6 Months',
'Field:Women and Girls Severity Assessment',
'Field:DAPEV Info Rating',
'Field:Canadian Sponsorship Info Rating',
'Field:Aggr 1 Reason Unable to Move',
'Field:Nationality2',
'Field:Email Address1',
'Field:I 130 F',
'Field:Worked for USG',
'Field:Aggr 1 Mst Svr Only Threats',
'Field:German Reunification Info Rating',
'Field:Traumatic Incident 1 Assessment',
'Field:Aggressor 2 Score',
'Field:Medical Condition 1 Assessment',
'Field:Medical Condition 1 Score',
'Field:Medical Condition 2 Score',
'Field:Medical Condition 3 Score',
'Field:Medical Condition 2 Assessment',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Mst Svr Attack',
'Field:USRAP Info Rating',
'Field:Aggr 2 SVT Rating',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggressor 3 Score',
'Field:Aggr 2 Mst Svr Attack']:
    changeto_binary_field(full_clean,i)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: Updated to binary null/not null
Reason Unable to Move: من الماده                                                                                                                          1
Reason Unable to Move: ولك خلص                                                                                                                            1
Reason Unable to Move: صاحب المنزل الو معي اجار ٣ اشهر                                                                                                    1
Reason Unable to Move: احوالنا السيئة وصاحب البيت بدو منا اجار بيت                                                                                        1
Reason Unable to Move: بسبب  رخص  البيت                                                                                                                   1
Reason Unable to Move: مامعي مصاري روح اخد بيت هون بيوت غالي ومافي وين بدي روح                                                                            1
Reason Unable to Move: عد

[log]: Updated to binary null/not null
5    19
1    18
3    13
4    10
2    10
١     1
٥     1
Name: Field:Women and Girls Severity Assessment, dtype: int64
0    16411
1       72
Name: Field:Women and Girls Severity Assessment, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5           18
1           12
3           11
4            7
2            5
انا طالب     1
0            1
Name: Field:DAPEV Info Rating, dtype: int64
0    16428
1       55
Name: Field:DAPEV Info Rating, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5                                                           630
3                                                           331
1                                                           254
4                                                           198
2                                                           168
١                                                             7
لم افهم             

[log]: Updated to binary null/not null
Attack Severity: 1    171
Attack Severity: 3    140
Attack Severity: 5    120
Attack Severity: 2     81
Attack Severity: 4     62
1                      31
5                      29
3                      25
2                      15
4                       9
٢                       3
١                       1
٥                       1
Name: Field:Aggr 1 Attack Within Last 6 Months, dtype: int64
0    15795
1      688
Name: Field:Aggr 1 Attack Within Last 6 Months, dtype: int64
------------------------------
[log]: Updated to binary null/not null
5.0    328
4.0     92
3.0     81
1.0     73
2.0     39
Name: Field:Traumatic Incident 2 Assessment, dtype: int64
0    15870
1      613
Name: Field:Traumatic Incident 2 Assessment, dtype: int64
------------------------------
[log]: Updated to binary null/not null
Severity of Most Serious Attack: 5    169
Severity of Most Serious Attack: 1    110
Severity of Most Serious Attack: 3    103
Severity of Most Ser

In [27]:
def convert_arabic_boolean(df, column_name, as_yes, as_no):
    if as_yes not in df[column_name].values and as_no not in df[column_name].values:
        print("[log]: no change")
        return
#     new = df[column_name].apply(lambda x: "Yes" if x==as_yes else ("No" if x==as_no else (np.nan if x!="Yes" and x!="No" else x)))
    new = df[column_name].apply(lambda x: "Yes" if x==as_yes else ("No" if x==as_no else np.nan))

    df[column_name] = new
    print("[log]: converted "+column_name)

In [28]:
arrabic_no = "لا"
arrabic_yes = "نعم"

In [29]:
for i in ['Field:Medical Condition 3 Inc Treatment',
'Field:Medical Condition 3 SVT',
'Field:Medical Condition 3 Treatment Abroad',
'Field:Traumatic Incident 2 Ong Mdl Needs',
'Field:Medical Condition 2 Inc Treatment',
'Field:Medical Condition 2 SVT',
'Field:Medical Condition 2 Treatment Abroad',
'Field:Medical Condition 1 Inc Treatment',
'Field:Medical Condition 1 SVT',
'Field:Medical Condition 1 Treatment Abroad',
'Field:Traumatic Incident 1 Ong Mdl Needs']:
    convert_arabic_boolean(full_clean,i,arrabic_yes,arrabic_no)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:Medical Condition 3 Inc Treatment
لا     258
نعم    108
No       2
Yes      1
Name: Field:Medical Condition 3 Inc Treatment, dtype: int64
No     258
Yes    108
Name: Field:Medical Condition 3 Inc Treatment, dtype: int64
------------------------------
[log]: converted Field:Medical Condition 3 SVT
لا             224
نعم            141
Yes              2
No               1
نعم نعم نعم      1
Name: Field:Medical Condition 3 SVT, dtype: int64
No     224
Yes    141
Name: Field:Medical Condition 3 SVT, dtype: int64
------------------------------
[log]: converted Field:Medical Condition 3 Treatment Abroad
نعم            268
لا أعلم         93
لا               4
Yes              3
نعم نعم نعم      1
Name: Field:Medical Condition 3 Treatment Abroad, dtype: int64
Yes    268
No       4
Name: Field:Medical Condition 3 Treatment Abroad, dtype: int64
------------------------------
[log]: converted Field:Traumatic Incident 2 Ong Mdl Needs
لا     536
نعم    248
No       2
Name: 

In [30]:
# convert null
def convert_to(df, column_name, not_null_as, null_as):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: not_null_as if not pd.isnull(x) else null_as)
    df[column_name] = new
    print("[log]: converted "+column_name)

In [31]:
for i in ['Field:I 130 IR',
'Field:SIV Follow to Join',
'Field:P 3',
'Field:SIV',
'Field:DAP via EV',
# 'Field:DAP as Relative of SIV',
'Field:EO Waiver',
'Field:SIV 101a',
'Field:SIV 1059',
'Field:DAP via I 130',
'Field:ICORN CPJ',
'Field:Canadian Private Sponsorship',
'Field:Germany Family Reunification']:
    convert_to(full_clean,i,"Eligible","Not Eligible")
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:I 130 IR
I-130 IR    2
Name: Field:I 130 IR, dtype: int64
Not Eligible    16481
Eligible            2
Name: Field:I 130 IR, dtype: int64
------------------------------
[log]: converted Field:SIV Follow to Join
SIV Follow to Join    4
Name: Field:SIV Follow to Join, dtype: int64
Not Eligible    16479
Eligible            4
Name: Field:SIV Follow to Join, dtype: int64
------------------------------
[log]: converted Field:P 3
P-3    12
Name: Field:P 3, dtype: int64
Not Eligible    16471
Eligible           12
Name: Field:P 3, dtype: int64
------------------------------
[log]: converted Field:SIV
SIV    15
Name: Field:SIV, dtype: int64
Not Eligible    16468
Eligible           15
Name: Field:SIV, dtype: int64
------------------------------
[log]: converted Field:DAP via EV
DAP via EV    33
Name: Field:DAP via EV, dtype: int64
Not Eligible    16450
Eligible           33
Name: Field:DAP via EV, dtype: int64
------------------------------
[log]: converted Field:EO Waiver
E

In [32]:
for i in ['Field:Aggr 3 Threats Since Moving',
'Field:RFR Submission',
'Field:US Prior Admission',
'Field:Aggr 3 Moved to Avoid Aggr',
'Field:Testing Only',
'Field:Family Worked for USG',
'ILI Assistance Provided ',
'Field:RFR',
'Field:Aggr 2 Moved to Avoid Aggr',
'Flags',
'Field:Aggr 1 Threats Since Moving',
'Field:USRAP Access',
'Field:Aggr 1 Moved to Avoid Aggr',
'Field:Multiple Incidents',
'Field:Vulnerability Assessed',
'URN:Telegram',
'Field:Lack of Durable Solutions',
'Field:Women and Girls at Risk',
'Field:SVT',
'Field:LGBT',
'Field:Hired Lawyer',
'Field:Pause Handoffs',
'Field:Prescreening Complete',
'Field:Ongoing USRAP',
'Field:Previous US Resettlement',
'URN:Facebook']:
    convert_to(full_clean,i,"Yes","No")
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

[log]: converted Field:Aggr 3 Threats Since Moving
Received Threats Since Moving: Yes    37
Received Threats Since Moving: No      3
Name: Field:Aggr 3 Threats Since Moving, dtype: int64
No     16443
Yes       40
Name: Field:Aggr 3 Threats Since Moving, dtype: int64
------------------------------
[log]: converted Field:RFR Submission
RFR Submission    19
Name: Field:RFR Submission, dtype: int64
No     16464
Yes       19
Name: Field:RFR Submission, dtype: int64
------------------------------
[log]: converted Field:US Prior Admission
RTD/RRV    64
Name: Field:US Prior Admission, dtype: int64
No     16419
Yes       64
Name: Field:US Prior Admission, dtype: int64
------------------------------
[log]: converted Field:Aggr 3 Moved to Avoid Aggr
Moved to Avoid This Aggressor: Unable    50
Moved to Avoid This Aggressor: Yes       40
Moved to Avoid This Aggressor: No        13
Name: Field:Aggr 3 Moved to Avoid Aggr, dtype: int64
No     16380
Yes      103
Name: Field:Aggr 3 Moved to Avoid Aggr, 

In [33]:
convert_to(full_clean,'Field:RFR within deadline',"Yes","Not Applicable")

[log]: converted Field:RFR within deadline


In [34]:
# convert 'Field:Aggr 2 Threats Since Moving'
def convert_threats(df, column_name='Field:Aggr 2 Threats Since Moving'):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: "Yes" if x=="Received Threats Since Moving: Yes" else ("No" if x == "Received Threats Since Moving: No" else "Not Applicable"))
    df[column_name] = new
    print("[log]: converted "+column_name)

In [35]:
convert_threats(full_clean)
print(full['Field:Aggr 2 Threats Since Moving'].value_counts())
print(full_clean['Field:Aggr 2 Threats Since Moving'].value_counts())

[log]: converted Field:Aggr 2 Threats Since Moving
Received Threats Since Moving: Yes    102
Received Threats Since Moving: No      40
Name: Field:Aggr 2 Threats Since Moving, dtype: int64
Not Applicable    16341
Yes                 102
No                   40
Name: Field:Aggr 2 Threats Since Moving, dtype: int64


In [36]:
full['Vulnerability Threshold'].value_counts()

Below             2171
Not Applicable      98
Above               48
-                    1
Name: Vulnerability Threshold, dtype: int64

In [37]:
full_clean['Vulnerability Threshold'] = full['Vulnerability Threshold'].apply(lambda x: np.nan if x!="Below" and x!="Above" else x)

In [38]:
# convert 'Field:End Feedback Follow Up Conversation'
def convert_follow_up(df, column_name='Field:End Feedback Follow Up Conversation'):
    if df[column_name].isna().sum() == 0:
        print("[log]: no change")
        return
    new = df[column_name].apply(lambda x: x if not pd.isnull(x) else "Not Applicable")
    df[column_name] = new
    print("[log]: converted "+column_name)

In [39]:
convert_follow_up(full_clean)

[log]: converted Field:End Feedback Follow Up Conversation


In [40]:
full_clean = full_clean[full_clean['Field:Testing Only']=="No"]
full_clean = full_clean.drop(columns=['Field:Testing Only'])

In [41]:
def create_adult_field(df, column_name='Field:Minor or Adult'):
    if "Adult" not in df.columns:
        df["Adult"] = df[column_name].apply(lambda x: 1 if x=="Adult" else 0)
        df.drop(columns=[column_name])
        print("Done")
    else:
        print("Already created Adult column")

In [42]:
create_adult_field(full_clean)

Done


In [43]:
def create_married_field(df, column_name='Field:Marital Status'):
    if "Married" not in df.columns:
        df["Married"] = df[column_name].apply(lambda x: 1 if x=="Married" else 0)
        df.drop(columns=[column_name])
        print("Done")
    else:
        print("Already created Married column")

In [44]:
create_married_field(full_clean)

Done


In [45]:
import re
def string_to_int(string):
    end_num = re.compile('(\d)\s*.*$')
    try:
        return(int(end_num.findall(string)[0]))
    except: return(np.nan)

In [46]:
def convert_to_numeric(df, column_name):
    new = df[column_name].apply(lambda x: string_to_int(str(x)))
    df[column_name] = new

In [47]:
for i in ['Field:USRAP Info Rating',
'Field:Aggr 1 Mst Svr Attack',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:Medical Condition 2 Assessment',
'Field:Traumatic Incident 1 Assessment',
# 'Field:Lebanon UNHCR Info Rating',
'Field:German Reunification Info Rating',
'Field:Aggr 1 Mst Svr Threat',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 SVT Rating',
'Field:Aggr 3 SVT Rating',
'Field:Aggressor 1 Score',
'Field:Aggr 1 SVT Rating',
'Field:Medical Condition 3 Score',
'Field:Medical Condition 2 Score',
'Field:Medical Condition 1 Assessment',
'Field:Medical Condition 1 Score',
'Field:Aggressor 2 Score',
'Field:Years in Current Country',
'Field:Medical Needs Score',
'Field:End Feedback Recommend Rating',
'Field:End Feedback Helpfulness Rating',
'Field:Age',
'Field:Aggr 2 Mst Svr Attack',
'Field:Aggressor 3 Score']:
    convert_to_numeric(full_clean,i)
    print(full[i].value_counts())
    print(full_clean[i].value_counts())
    print("-"*30)

5                                                                                               182
1                                                                                                71
3                                                                                                59
4                                                                                                56
2                                                                                                35
0                                                                                                 2
لايوجد جديد                                                                                       1
١                                                                                                 1
بس  انا ما دقولي  كرمال السفر انا قدمت وما ردولي خبر ... اذا فيكن تساعدوني وتقدمولي  طلب سفر      1
👍                                                                                                 1


Aggressor SVT Rating: 0    171
Aggressor SVT Rating: 5    107
Aggressor SVT Rating: 3     38
0                           34
Aggressor SVT Rating: 4     32
Aggressor SVT Rating: 1     26
5                           22
Aggressor SVT Rating: 2     12
3                            9
4                            3
2                            3
1                            1
Name: Field:Aggr 2 SVT Rating, dtype: int64
0    16000
1      458
Name: Field:Aggr 2 SVT Rating, dtype: int64
------------------------------
Aggressor SVT Rating: 5    31
Aggressor SVT Rating: 0    29
Aggressor SVT Rating: 3    14
Aggressor SVT Rating: 4    13
Aggressor SVT Rating: 1     9
Aggressor SVT Rating: 2     7
5                           4
0                           4
3                           3
Name: Field:Aggr 3 SVT Rating, dtype: int64
5.0    35
0.0    33
3.0    17
4.0    13
1.0     9
2.0     7
Name: Field:Aggr 3 SVT Rating, dtype: int64
------------------------------
6.0    724
3.0    242
1.0    228
4.0  

### Output cleaned data

In [48]:
cleaned = full_clean[['process_days','process_day_>=1','Vulnerability Threshold',
'Field:Aggr 3 Threats Since Moving',
'Field:RFR Submission',
'Field:US Prior Admission',
'Field:Aggr 3 Moved to Avoid Aggr',
'Field:Family Worked for USG',
'ILI Assistance Provided ',
'Field:RFR',
'Field:Aggr 2 Moved to Avoid Aggr',
'Flags',
'Field:Aggr 1 Threats Since Moving',
'Field:USRAP Access',
'Field:Aggr 1 Moved to Avoid Aggr',
'Field:Multiple Incidents',
'Field:Vulnerability Assessed',
'URN:Telegram',
'Field:Lack of Durable Solutions',
'Field:Women and Girls at Risk',
'Field:SVT',
'Field:LGBT',
'Field:Hired Lawyer',
'Field:Pause Handoffs',
'Field:Prescreening Complete',
'Field:Ongoing USRAP',
'Field:Previous US Resettlement',
'URN:Facebook',
'Field:RFR within deadline',
'Field:Aggr 2 Threats Since Moving',
'Field:End Feedback Follow Up Conversation',
'Field:I 130 IR',
'Field:SIV Follow to Join',
'Field:P 3',
'Field:SIV',
'Field:DAP via EV',
# 'Field:DAP as Relative of SIV',
'Field:EO Waiver',
'Field:SIV 101a',
'Field:SIV 1059',
'Field:DAP via I 130',
'Field:ICORN CPJ',
'Field:Canadian Private Sponsorship',
'Field:Germany Family Reunification',
'Field:Medical Condition 3 Inc Treatment',
'Field:Medical Condition 3 SVT',
'Field:Medical Condition 3 Treatment Abroad',
'Field:Traumatic Incident 2 Ong Mdl Needs',
'Field:Medical Condition 2 Inc Treatment',
'Field:Medical Condition 2 SVT',
'Field:Medical Condition 2 Treatment Abroad',
'Field:Medical Condition 1 Inc Treatment',
'Field:Medical Condition 1 SVT',
'Field:Medical Condition 1 Treatment Abroad',
'Field:Traumatic Incident 1 Ong Mdl Needs',
'Field:Handoff Timestamp',
'Date of Screening Decision',
'Created On',
'Field:USRAP Info Rating',
'Field:Aggr 1 Mst Svr Attack',
'Field:Traumatic Incident 2 Assessment',
'Field:Aggr 1 Attack Within Last 6 Months',
'Field:ICORN Info Rating',
'Field:Aggr 1 Threat Within Last 6 Months',
'Field:Medical Condition 2 Assessment',
'Field:Traumatic Incident 1 Assessment',
# 'Field:Lebanon UNHCR Info Rating',
'Field:German Reunification Info Rating',
'Field:Aggr 1 Mst Svr Threat',
'Field:Aggr 2 Mst Svr Only Threats',
'Field:Aggr 2 Attack Within Last 6 Months',
'Field:Aggr 2 Threat Within Last 6 Months',
'Field:Medical Condition 3 Assessment',
'Field:Aggr 2 SVT Rating',
'Field:Aggr 3 SVT Rating',
'Field:Aggressor 1 Score',
'Field:Aggr 1 SVT Rating',
'Field:Years in Current Country',
'Field:Medical Needs Score',
'Field:End Feedback Recommend Rating',
'Field:End Feedback Helpfulness Rating',
'Field:Age',
'Field:Aggr 2 Mst Svr Attack',
'Field:Aggressor 3 Score',
'Adult',
'Married',
'Client Location (Country)',
'Client Nationality',
'Field:Current Country',
'Field:Nationality1',
'Field:Gender',
'Language',
'Field:Aggr 3 Reason Unable to Move',
'Field:I130 Info Rating',
'Field:Aggr 3 Mst Svr Threat',
'Field:DAPI130 Info Rating',
'Field:Aggr 2 Mst Svr Threat',
'Field:Aggr 3 Mst Svr Only Threats',
'Field:P3 Info Rating',
'Field:Aggr 3 Mst Svr Attack',
'Field:Aggr 3 Attack Within Last 6 Months',
'Field:Aggr 2 Reason Unable to Move',
'Field:Aggr 3 Threat Within Last 6 Months',
'Field:Women and Girls Severity Assessment',
'Field:DAPEV Info Rating',
'Field:Canadian Sponsorship Info Rating',
'Field:Aggr 1 Reason Unable to Move',
'Field:Nationality2',
'Field:Email Address1',
'Field:I 130 F',
'Field:Worked for USG',
'Field:Aggr 1 Mst Svr Only Threats',
# 'Medical_X_LPPN',
'Field:LPPN Score',
'Screening Decision',
'Date of Screening Start',
'Program Eligibilities',
'Date of Referral',
'First Referral',
'In IRAP Process']]

In [49]:
null_count = pd.Series.to_frame(cleaned.isna().sum())
# null_count.rename(columns={"0": "null_count"})
null_count.columns = ["null_counts"]
null_count["total_counts"] = cleaned.shape[0]
null_count["null_percent"] = np.round(null_count["null_counts"] / cleaned.shape[0] , 5)
null_count["dtype"] = pd.Series.to_frame(cleaned.dtypes)
null_count["sample_value"] = cleaned.mode()[:1].T
null_count.sort_values(['null_counts'],ascending = False)[:20]

,null_counts,total_counts,null_percent,dtype,sample_value
Field:Aggr 1 Mst Svr Threat,16345,16458,0.99313,float64,5
Field:Aggr 3 SVT Rating,16344,16458,0.99307,float64,5
Field:Medical Condition 3 Treatment Abroad,16186,16458,0.98347,object,Yes
Field:Medical Condition 3 SVT,16093,16458,0.97782,object,No
Field:Medical Condition 3 Inc Treatment,16092,16458,0.97776,object,No
Field:Medical Condition 2 Treatment Abroad,15844,16458,0.96269,object,Yes
Field:Traumatic Incident 2 Ong Mdl Needs,15674,16458,0.95236,object,No
Field:Medical Condition 2 SVT,15603,16458,0.94805,object,No
Field:Medical Condition 2 Inc Treatment,15603,16458,0.94805,object,No
Field:Medical Condition 1 Treatment Abroad,15421,16458,0.93699,object,Yes


In [50]:
cleaned.to_csv("may_cleaned_data.csv")

### Import data and attempt to train classifier

In [51]:
data = pd.read_csv("may_cleaned_data.csv")
data = data.drop(columns=['Unnamed: 0','Program Eligibilities','Client Location (Country)','Client Nationality'])

/Users/xiaoxiangma/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (3,44,45,46,47,48,49,50,51,52,53,54,55,56,85,86,112,113,114,115,116,117) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [52]:
print(data.shape)
data.head()

(16458, 114)


,process_days,process_day_>=1,Vulnerability Threshold,Field:Aggr 3 Threats Since Moving,Field:RFR Submission,Field:US Prior Admission,Field:Aggr 3 Moved to Avoid Aggr,Field:Family Worked for USG,ILI Assistance Provided,Field:RFR,...,Field:Email Address1,Field:I 130 F,Field:Worked for USG,Field:Aggr 1 Mst Svr Only Threats,Field:LPPN Score,Screening Decision,Date of Screening Start,Date of Referral,First Referral,In IRAP Process
0,NaN,NaN,NaN,No,No,No,No,No,No,No,...,0,0,0,0,NaN,NaN,NaN,2020-03-01 00:00:00,No,Yes
1,NaN,NaN,NaN,No,No,No,No,No,No,No,...,0,0,0,0,NaN,NaN,NaN,2020-02-29 00:00:00,No,Yes
2,NaN,NaN,NaN,No,No,No,No,No,No,No,...,0,0,0,0,NaN,NaN,NaN,2020-02-29 00:00:00,No,Yes
3,NaN,NaN,NaN,No,No,No,No,No,No,No,...,0,0,0,0,NaN,NaN,NaN,2020-02-29 00:00:00,No,Yes
4,NaN,NaN,NaN,No,No,No,No,No,No,No,...,0,0,0,0,NaN,NaN,NaN,2020-02-28 00:00:00,No,Yes


In [53]:
train_data = data.dropna(subset=['process_days'])

In [54]:
null_count = pd.Series.to_frame(train_data.isna().sum())
# null_count.rename(columns={"0": "null_count"})
null_count.columns = ["null_counts"]
null_count["total_counts"] = train_data.shape[0]
null_count["null_percent"] = np.round(null_count["null_counts"] / train_data.shape[0] , 5)
null_count["dtype"] = pd.Series.to_frame(train_data.dtypes)
null_count["sample_value"] = train_data.mode()[:1].T
null_count.sort_values(['null_counts'],ascending = False)[:20]

,null_counts,total_counts,null_percent,dtype,sample_value
Field:Aggr 1 Mst Svr Threat,2597,2611,0.99464,float64,5
Field:Aggr 3 SVT Rating,2592,2611,0.99272,float64,0
Field:Medical Condition 3 Treatment Abroad,2565,2611,0.98238,object,Yes
Field:Medical Condition 3 Inc Treatment,2544,2611,0.97434,object,No
Field:Medical Condition 3 SVT,2544,2611,0.97434,object,No
Field:Medical Condition 2 Treatment Abroad,2503,2611,0.95864,object,Yes
Field:Medical Condition 2 Inc Treatment,2457,2611,0.94102,object,No
Field:Medical Condition 2 SVT,2457,2611,0.94102,object,No
Field:Traumatic Incident 2 Ong Mdl Needs,2446,2611,0.93681,object,No
Field:Medical Condition 1 Treatment Abroad,2442,2611,0.93527,object,Yes


In [55]:
train_data.shape

(2611, 114)

In [56]:
data['Vulnerability Threshold'].value_counts()

Below    2171
Above      48
Name: Vulnerability Threshold, dtype: int64

#### Drop high% Nan columns

In [57]:
train_data = train_data.drop(train_data.columns[train_data.isnull().mean()>0.7], axis=1).dropna()
train_data.shape

(2177, 87)

In [58]:
train_data.replace(dict.fromkeys(['Yes','Eligible','Above'], 1),inplace=True)
train_data.replace(dict.fromkeys(['No','Not Eligible','Below','Not Applicable'], 0),inplace=True)

train_data.head()

,process_days,process_day_>=1,Vulnerability Threshold,Field:Aggr 3 Threats Since Moving,Field:RFR Submission,Field:US Prior Admission,Field:Aggr 3 Moved to Avoid Aggr,Field:Family Worked for USG,ILI Assistance Provided,Field:RFR,...,Field:Nationality2,Field:Email Address1,Field:I 130 F,Field:Worked for USG,Field:Aggr 1 Mst Svr Only Threats,Screening Decision,Date of Screening Start,Date of Referral,First Referral,In IRAP Process
18,6.0,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Cannot Represent,2020-04-03,2020-02-28 00:00:00,0,0
25,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Cannot Represent,2020-03-05,2020-02-29 00:00:00,0,0
32,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Cannot Represent,2020-03-12,2020-02-27 00:00:00,1,0
33,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Cannot Represent,2020-03-13,2020-02-27 00:00:00,1,0
34,0.0,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,Cannot Represent,2020-03-03,2020-02-28 00:00:00,1,0


In [59]:
for col in train_data.columns:
    if len(train_data[col].unique()) == 1:
        train_data.drop(col,inplace=True,axis=1)

In [60]:
train_data.shape

(2177, 80)

In [61]:
# Convert string to datetime
for i in train_data.columns:
    if (train_data[i].dtype) == "object":
        try:
            train_data[i] = train_data[i].astype('datetime64')
            print("converted: "+i)
        except:
            pass

converted: Date of Screening Decision
converted: Date of Screening Start
converted: Date of Referral


In [62]:
# Convert datetime to ordinal
for i in train_data.columns:
    if (train_data[i].dtype) == "datetime64[ns]":
        try:
            train_data[i] = train_data[i].apply(lambda x: x.toordinal())
            print("converted: "+i)
        except:
            pass

converted: Date of Screening Decision
converted: Date of Screening Start
converted: Date of Referral


In [63]:
train_data.shape

(2177, 80)

In [64]:
for i in train_data.columns:
    if (train_data[i].dtype) == "object":
        train_data = train_data.drop(columns=[i])

In [65]:
train_data.shape

(2177, 79)

### Ready for training

In [66]:
for i in ["Date of Referral",
'First Referral',
'In IRAP Process',
'Date of Screening Start',
'ILI Assistance Provided ',
# 'Vulnerability Threshold',
'Date of Screening Decision',
'Flags']:
    train_data = train_data.drop(columns=[i])

In [67]:
train_data.shape

(2177, 72)

In [68]:
train_data["Vulnerability Threshold"].value_counts()

0    2133
1      44
Name: Vulnerability Threshold, dtype: int64

In [71]:
train_data.to_csv("derived data/new_data.csv")

In [72]:
x = train_data.drop(columns=['process_days','process_day_>=1','Vulnerability Threshold'])
y = train_data["Vulnerability Threshold"]

In [73]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif, SelectKBest

In [74]:
def get_feat(method, k, x, y):
    select = SelectKBest(method, k)
    _ = select.fit_transform(x, y)
    return [x.columns[i] for i, val in enumerate(select.get_support()) if val] 

In [75]:
num_features = 3
chi2_select = get_feat(chi2, num_features, x, y)
f_select = get_feat(f_classif, num_features, x, y)
# mutual_select = get_feat(mutual_info_classif, num_features, x, y)


#### Statsmodel logistic regression is not a good fit, attempt sklearn in next round

In [77]:
for i in [chi2_select,f_select]:
    process_day7=sm.Logit(y,x[i]).fit(method="bfgs",maxiter=1000)
    print(process_day7.summary())

Optimization terminated successfully.
         Current function value: 0.688940
         Iterations: 22
         Function evaluations: 27
         Gradient evaluations: 27
                              Logit Regression Results                             
Dep. Variable:     Vulnerability Threshold   No. Observations:                 2177
Model:                               Logit   Df Residuals:                     2175
Method:                                MLE   Df Model:                            1
Date:                     Sun, 06 Sep 2020   Pseudo R-squ.:                  -5.969
Time:                             12:13:33   Log-Likelihood:                -1499.8
converged:                            True   LL-Null:                       -215.22
Covariance Type:                 nonrobust   LLR p-value:                     1.000
                                               coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------

/Users/xiaoxiangma/.local/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/xiaoxiangma/.local/lib/python3.7/site-packages/statsmodels/base/model.py:492: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
